<a href="https://colab.research.google.com/github/d2525456d/Doantotnghiep/blob/main/Thu%E1%BA%ADt%20to%C3%A1n%20m%C3%B4%20ph%E1%BB%8Fng%20luy%E1%BB%87n%20kim/toiuu_sanluong%E2%AD%90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import random
import pandas as pd

# Load the uploaded file
file_path = '/content/data.csv'
df = pd.read_csv(file_path)

# Prepare data
tasks_times = df['TGLT'].tolist()
machines = [
    0, 0, 0, 0, 1, 2, 1, 1, 0, 2, 7, 1, 7, 0, 1, 1, 2, 1, 2, 2, 1, 1, 1, 3, 2, 4, 1,
    5, 1, 1, 1, 1, 2, 7, 1, 1, 1, 1, 1, 2, 1, 1, 3, 2, 2, 7, 1, 1, 3, 7,
    6, 2, 0, 0, 0, 2, 1, 2, 4, 2, 4, 1, 1, 0, 1, 1, 1, 4, 0, 1, 1, 2, 0, 3, 0, 3, 1, 0, 0, 0, 0, 0, 1, 0, 0
]
df["Thiết bị"] = machines
proficiency_levels = {
    "Lê Thị Phương Thảo": 1.48, "Hoàng Thị Mỹ Yến": 1.16, "Trần Thị Tuyền": 1.48,
    "Ngô Khắc Hiển": 1.00, "Hồ Thị Diệp": 1.16, "Nguyễn Thị Mỹ Nhung": 1.16,
    "Thái Thị Nga": 1.16, "Trần Thị Cúc": 1.00, "Lê Thị Thanh Hà": 1.16,
    "Nguyễn Thị Thúy": 1.16, "Trần Thị Cẩm Rin": 1.32, "Hoàng Thị Nở": 1.16,
    "Lê Thị Như Quỳnh": 1.16, "Trần Thị Thương": 1.16, "Đinh Thị Mi": 1.16,
    "Nguyễn Thị Ngọc Lan": 1.16, "Bùi Thị Hồng Vân": 1.00, "Trần Thị Oanh": 1.16,
    "Nguyễn Thị Gấm": 1.32, "Phan Thị Thu Hoài": 1.16, "Nguyễn Thị Thu": 1.16,
    "Nguyễn Thị Hài": 1.16, "Nguyễn Thị Hồng": 1.48
}
# Thêm cột "Loại Máy" từ danh sách machines
df['Loại Máy'] = machines

# Thay thế cột "TÊN CÔNG ĐOẠN" bằng chuỗi kết hợp "TÊN CÔNG ĐOẠN" và "Loại Máy"
df['TÊN CÔNG ĐOẠN'] = df['TÊN CÔNG ĐOẠN'] + ' - ' + df['Loại Máy'].astype(str)
workers = list(proficiency_levels.keys())

def calculate_fitness_with_product(individual, tasks_times, proficiency_levels, df):
    detailed_task_allocation = {worker: [] for worker in workers}

    for task_idx, worker_list in enumerate(individual):
        task_time = tasks_times[task_idx]
        n_workers = len(worker_list)

        theoretical_time_per_worker = task_time / n_workers
        for worker in worker_list:
            proficiency = proficiency_levels[worker]
            worker_time = theoretical_time_per_worker / proficiency
            detailed_task_allocation[worker].append(worker_time)

    worker_time_summary = [
        sum(worker_times) for worker_times in detailed_task_allocation.values()
    ]
    max_worker_time = max(worker_time_summary)

    stage_times = []
    for task_idx, worker_list in enumerate(individual):
        task_time = tasks_times[task_idx]
        n_workers = len(worker_list)
        theoretical_time_per_worker = task_time / n_workers
        worker_times = [theoretical_time_per_worker / proficiency_levels[worker] for worker in worker_list]

        if n_workers == 1:
            total_time_for_task = worker_times[0]
        elif n_workers == 2:
            total_time_for_task = min(worker_times) * 2
        elif n_workers == 3:
            total_time_for_task = min(worker_times) * 3

        stage_times.append(total_time_for_task)

    total_1 = sum(stage_times[0:4])
    total_2 = sum(stage_times[4:27])
    total_3 = sum(stage_times[27:50])
    total_4 = sum(stage_times[50:53])
    total_5 = sum(stage_times[53:57])
    total_6 = sum(stage_times[57:85])

    re_stage_time = total_1 + max(max(total_2, total_3) + total_4, total_5) + total_6

    total_product = (28800 - re_stage_time) / max_worker_time + 1
    fitness = -total_product
    return fitness, total_product

def check_worker_machines(individual, machines):
    worker_machines = {worker: set() for worker in workers}
    for task_idx, worker_list in enumerate(individual):
        machine_type = machines[task_idx]
        for worker in worker_list:
            worker_machines[worker].add(machine_type)
    for i in list(worker_machines):
      if str(worker_machines[i]).replace("0,","").count(",") >= 2:
        return False
    return True

# Hàm kiểm tra có thỏa ràng buộc hay không
def is_valid_machine_assignment(worker, machine_type, worker_machine_assignment):
    assigned_machines = worker_machine_assignment[worker]
    non_zero_machines = {m for m in assigned_machines if m != 0}

    if machine_type == 0:
        return len(non_zero_machines) <= 2
    else:
        return len(non_zero_machines) < 2 or machine_type in assigned_machines

#Hàm trả về thiết bị công nhân sử dụng
def calculate_worker_machines(individual, machines):
    worker_machines = {worker: set() for worker in workers}
    for task_idx, worker_list in enumerate(individual):
        machine_type = machines[task_idx]
        for worker in worker_list:
            worker_machines[worker].add(machine_type)
    worker_machines_df = pd.DataFrame([
        {"Worker": worker, "Machines Used": sorted(list(machines_used))}
        for worker, machines_used in worker_machines.items()
    ]).sort_values(by="Worker").reset_index(drop=True)
    return worker_machines_df

# Hàm khởi tạo phân công công nhân
def greedy_worker_assignment_with_constraints(tasks_times, machines, proficiency_levels, df):
    sorted_workers = sorted(proficiency_levels.keys(), key=lambda x: proficiency_levels[x])
    individual = []
    worker_machine_assignment = {worker: set() for worker in sorted_workers}

    for task_idx, task_time in enumerate(tasks_times):
        machine_type = machines[task_idx]
        available_workers = [
            worker for worker in sorted_workers
            if is_valid_machine_assignment(worker, machine_type, worker_machine_assignment)
        ]

        if available_workers:
            n_selected_workers = random.randint(1, min(3, len(available_workers)))
            selected_workers = random.sample(available_workers, n_selected_workers)
            for worker in selected_workers:
                worker_machine_assignment[worker].add(machine_type)
            individual.append(selected_workers)
        else:
            # Nếu không còn công nhân hợp lệ, gán bất kỳ công nhân nào (xảy ra khi bị thiếu ràng buộc)
            fallback_worker = random.choice(sorted_workers)
            individual.append([fallback_worker])
            worker_machine_assignment[fallback_worker].add(machine_type)

    return individual

# Chạy thuật toán mô phỏng luyện kim
def simulated_annealing_with_constraints(
    initial_solution, tasks_times, proficiency_levels, df, initial_temp=1000, cooling_rate=0.95, max_iter=100000
):
    worker_machine_assignment = {worker: set() for worker in workers}

    current_solution = initial_solution
    current_fitness, current_product = calculate_fitness_with_product(current_solution, tasks_times, proficiency_levels, df)
    best_solution = current_solution
    best_fitness = current_fitness
    best_product = current_product
    temperature = initial_temp

    for iteration in range(max_iter):
        new_solution = current_solution[:]
        task_idx = random.randint(0, len(tasks_times) - 1)
        machine_type = machines[task_idx]

        # Lựa chọn công nhân phù hợp
        possible_workers = [
            worker for worker in workers
            if is_valid_machine_assignment(worker, machine_type, worker_machine_assignment)
        ]
        n_selected_workers = random.randint(1, min(3, len(possible_workers))) if possible_workers else 1
        new_worker_list = random.sample(possible_workers, n_selected_workers) if possible_workers else [random.choice(workers)]

        # Gán giải pháp mới
        new_solution[task_idx] = new_worker_list
        for worker in new_worker_list:
            worker_machine_assignment[worker].add(machine_type)
        if check_worker_machines(new_solution, machines) == False:
            continue
        # Tính fitness cho giải pháp mới
        new_fitness, new_product = calculate_fitness_with_product(new_solution, tasks_times, proficiency_levels, df)
        delta_fitness = new_fitness - current_fitness

        # Chấp nhận giải pháp mới
        if delta_fitness < 0 or np.exp(-delta_fitness / max(temperature, 1e-6)) > random.random():
            current_solution = new_solution
            current_fitness = new_fitness
            current_product = new_product

        # Cập nhật giải pháp tốt nhất
        if current_fitness < best_fitness:
            best_solution = current_solution
            best_fitness = current_fitness
            best_product = current_product
            print(best_product)

        # Giảm nhiệt độ
        temperature *= cooling_rate

    return best_solution, -best_fitness, best_product

# Khởi tạo giải pháp ban đầu
initial_solution = greedy_worker_assignment_with_constraints(tasks_times, machines, proficiency_levels, df)
while check_worker_machines(initial_solution, machines)==False:
  initial_solution = greedy_worker_assignment_with_constraints(tasks_times, machines, proficiency_levels, df)

# Thực hiện tối ưu bằng thuatah toán mô phỏng luyện kim
best_solution, best_fitness, best_product = simulated_annealing_with_constraints(
    initial_solution, tasks_times, proficiency_levels, df
)
print("Sản lượng tốt nhất đạt được:", best_fitness)

In [ ]:
initial_solution = greedy_worker_assignment_with_constraints(tasks_times, machines, proficiency_levels, df)
while check_worker_machines(initial_solution, machines)==False:
  initial_solution = greedy_worker_assignment_with_constraints(tasks_times, machines, proficiency_levels, df)

# Optimize using Simulated Annealing with constraints
best_solution, best_fitness, best_product = simulated_annealing_with_constraints(
    initial_solution, tasks_times, proficiency_levels, df
)

127.07731481106083
127.13251836199193
127.13491851638024
127.14561931280986
127.16050027001737
130.16794246824014
130.17040279546367
130.17803911920282
130.18593876445024
130.20070072779137
132.94139288962492
134.76867493926315
136.47636015125377
136.47976440657294
136.51566382630213
136.51721121508356
136.52082178890691
140.0715856785726
142.81276217674122
143.1201843223631
143.12211455258324
157.69910495774317
157.71421380960345
157.72256640860414
160.58701588152272
160.5924835493801
160.59552114263423
166.26670438340153
167.28284899319382
167.28697737349773
167.3398996652655
167.36024056365517
167.39414206097132
169.44839072839073
170.17623902346665
171.85962859265183
174.49778816220234
174.50637539694762
175.44869433318746
175.48455983754613
177.3763147317277
177.3888183496727
177.393518275726
177.4003594808306
177.4506613921036
177.45804699018734
179.0026032080178
200.44995274717948
200.45292849483607
209.2812013321937
209.30023573290745
210.70737319182686
210.73532197756543
210.7

In [ ]:
calculate_worker_machines(best_solution, machines)

,Worker,Machines Used
0,Bùi Thị Hồng Vân,"[0, 1, 3]"
1,Hoàng Thị Mỹ Yến,"[1, 4]"
2,Hoàng Thị Nở,"[2, 4]"
3,Hồ Thị Diệp,"[1, 7]"
4,Lê Thị Như Quỳnh,"[0, 2, 3]"
5,Lê Thị Phương Thảo,"[4, 7]"
6,Lê Thị Thanh Hà,"[0, 1, 7]"
7,Nguyễn Thị Gấm,"[0, 2, 6]"
8,Nguyễn Thị Hài,"[0, 1, 3]"
9,Nguyễn Thị Hồng,"[0, 1, 5]"


In [ ]:
def display_worker_tasks(individual):
    # Tạo một dictionary lưu trữ các công đoạn mà mỗi công nhân đảm nhiệm
    worker_tasks = {worker: [] for worker in workers}

    # Lặp qua từng công đoạn trong cá thể tốt nhất
    for task_idx, worker_list in enumerate(individual):
        # Lấy tên công đoạn và loại máy cho công đoạn hiện tại
        task_name = df['TÊN CÔNG ĐOẠN'].iloc[task_idx]  # Sử dụng iloc để tránh lỗi index
        machine_type = machines[task_idx]

        # Thêm công đoạn và loại máy vào danh sách công việc của từng công nhân
        for worker in worker_list:
            worker_tasks[worker].append(f"{task_name} (Máy {machine_type})")

    # Hiển thị kết quả
    for worker, tasks in worker_tasks.items():
        if tasks:  # Nếu công nhân có công việc
            tasks_str = ", ".join(tasks)
            print(f"Công nhân {worker} làm các công đoạn: {tasks_str}")
        else:
            print(f"Công nhân {worker} không có công việc.")

display_worker_tasks(best_solution)


Công nhân Lê Thị Phương Thảo làm các công đoạn: Lựa số+ bỏ rập đh tt + lấy dấu + paget - 7 (Máy 7), Chạy kansai lưng - 4 (Máy 4)
Công nhân Hoàng Thị Mỹ Yến làm các công đoạn: Kẹp mí đáp túi + bỏ dây treo thân sau + đo gọt đây treo + lấy dấu đáp túi - 1 (Máy 1), Mí túi thân sau - 1 (Máy 1), Ghim miêng túi thân sau vào đáp túi - 1 (Máy 1), Kansai đáy thân sau - 4 (Máy 4), Lựa số đóng túi nhỏ vào bao túi - 1 (Máy 1), Lược bao túi vào thân x2 - 1 (Máy 1), Kansai 2 kim ssays thân trước - 1 (Máy 1)
Công nhân Trần Thị Tuyền làm các công đoạn: Xã phà thun lưng - 0 (Máy 0), Đo cắt pat túi thân sau - 0 (Máy 0), Đo cắt dây treo gắn túi thân sau - 0 (Máy 0), Cuốn miệng túi đồng hồ - 1 (Máy 1), Vs3c xq túi đồng hồ - 2 (Máy 2), Mí âm miêng túi hông x2 - 1 (Máy 1), Lược bao túi vào thân x2 - 1 (Máy 1), Vs3c bao túi hông x2+lộn - 2 (Máy 2), Vs3c paget tròn - 2 (Máy 2), Bấm góc vs3c đáy tt - 2 (Máy 2), Kansai 2 kim ssays thân trước - 1 (Máy 1), Nối thun lưng + cắt - 2 (Máy 2), Lựa số + vs4c sườn thân x

In [ ]:
calculate_fitness_with_product(best_solution, tasks_times, proficiency_levels, df)

(-291.73078817733995, 291.73078817733995)

In [ ]:
def display_task_allocation_with_time(individual):
    detailed_task_allocation = {worker: [] for worker in workers}

    for task_idx, worker_list in enumerate(individual):
        task_time = df['TGLT'][task_idx]
        n_workers = len(worker_list)

        # Tính thời gian chia đều lý thuyết cho mỗi công nhân
        theoretical_time_per_worker = task_time / n_workers

        # Điều chỉnh thời gian cho từng công nhân theo độ thông thạo của họ
        for worker in worker_list:
            proficiency = proficiency_levels[worker]
            worker_time = theoretical_time_per_worker / proficiency  # Điều chỉnh thời gian theo độ thông thạo
            detailed_task_allocation[worker].append(f"Công đoạn {df['TÊN CÔNG ĐOẠN'].iloc[task_idx]}: {worker_time:.2f} giây")

    return detailed_task_allocation


# Chọn cá thể tốt nhất (cá thể có độ lệch chuẩn nhỏ nhất)
task_allocation_with_time = display_task_allocation_with_time(best_solution)

# Hiển thị chi tiết công việc cho mỗi công nhân
worker_task_summary = []
worker_time = []

for worker, tasks in task_allocation_with_time.items():
    # Tính tổng thời gian cho mỗi công nhân
    total_time = sum(float(task.split(": ")[1].split()[0]) for task in tasks)
    # Chi tiết các công đoạn
    task_details = " + ".join(tasks)
    worker_task_summary.append({
        'Công Nhân': worker,
        'Chi Tiết Công Việc': task_details,
        'Tổng Thời Gian': total_time
    })
    worker_time.append({
        'Công Nhân': worker,
        'Tổng Thời Gian': total_time
    })

# Chuyển thành DataFrame để hiển thị
worker_task_summary_df = pd.DataFrame(worker_task_summary)

# Hiển thị kết quả chi tiết cho mỗi công nhân
worker_task_summary_df
def display_task_allocation_with_time(individual):
    detailed_task_allocation = {worker: [] for worker in workers}

    for task_idx, worker_list in enumerate(individual):
        task_time = df['TGLT'][task_idx]
        n_workers = len(worker_list)

        # Tính thời gian chia đều lý thuyết cho mỗi công nhân
        theoretical_time_per_worker = task_time / n_workers

        # Điều chỉnh thời gian cho từng công nhân theo độ thông thạo của họ
        for worker in worker_list:
            proficiency = proficiency_levels[worker]
            worker_time = theoretical_time_per_worker / proficiency  # Điều chỉnh thời gian theo độ thông thạo
            detailed_task_allocation[worker].append(f"Công đoạn {df['TÊN CÔNG ĐOẠN'].iloc[task_idx]}: {worker_time:.2f} giây")

    return detailed_task_allocation


# Chọn cá thể tốt nhất (cá thể có độ lệch chuẩn nhỏ nhất)
task_allocation_with_time = display_task_allocation_with_time(best_solution)

# Hiển thị chi tiết công việc cho mỗi công nhân
worker_task_summary = []
worker_time = []

for worker, tasks in task_allocation_with_time.items():
    # Tính tổng thời gian cho mỗi công nhân
    total_time = sum(float(task.split(": ")[1].split()[0]) for task in tasks)
    # Chi tiết các công đoạn
    task_details = " + ".join(tasks)
    worker_task_summary.append({
        'Công Nhân': worker,
        'Chi Tiết Công Việc': task_details,
        'Tổng Thời Gian': total_time
    })
    worker_time.append({
        'Công Nhân': worker,
        'Tổng Thời Gian': total_time
    })

# Chuyển thành DataFrame để hiển thị
worker_task_summary_df = pd.DataFrame(worker_task_summary)
worker_time_df =  pd.DataFrame(worker_time)
# Hiển thị kết quả chi tiết cho mỗi công nhân
worker_task_summary_df

,Công Nhân,Chi Tiết Công Việc,Tổng Thời Gian
0,Lê Thị Phương Thảo,Công đoạn Lựa số+ bỏ rập đh tt + lấy dấu + pag...,94.59
1,Hoàng Thị Mỹ Yến,Công đoạn Kẹp mí đáp túi + bỏ dây treo thân sa...,88.65
2,Trần Thị Tuyền,Công đoạn Xã phà thun lưng - 0: 1.01 giây + Cô...,92.23
3,Ngô Khắc Hiển,Công đoạn Chốt pat túi thân sau - 1: 7.00 giây...,85.34
4,Hồ Thị Diệp,Công đoạn Chốt dây treo túi sau - 1: 4.89 giây...,81.33
5,Nguyễn Thị Mỹ Nhung,Công đoạn May nhãn cơi túi thân sau - 1: 26.72...,89.08
6,Thái Thị Nga,Công đoạn Lấy dấu cắt gai xù túi sau + gọt góc...,89.96
7,Trần Thị Cúc,Công đoạn Bọ túi thân sau x2 - 3: 10.00 giây +...,82.50
8,Lê Thị Thanh Hà,Công đoạn Nối dây pat túi thân sau - 1: 8.62 g...,85.20
9,Nguyễn Thị Thúy,Công đoạn Lược miệng túi thân sau - 1: 5.46 gi...,89.37


In [ ]:
def display_stage_times(individual):
    stage_times = []

    for task_idx, worker_list in enumerate(individual):
        task_time = df['TGLT'][task_idx]
        n_workers = len(worker_list)

        # Tính thời gian lý thuyết chia đều cho mỗi công nhân
        theoretical_time_per_worker = task_time / n_workers

        # Tính thời gian điều chỉnh cho mỗi công nhân dựa trên độ thông thạo
        worker_times = []
        for worker in worker_list:
            proficiency = proficiency_levels[worker]
            worker_time = theoretical_time_per_worker / proficiency  # Điều chỉnh thời gian theo độ thông thạo
            worker_times.append(worker_time)

        # Nếu có 1 công nhân, giữ nguyên thời gian
        if n_workers == 1:
            total_time_for_task = worker_times[0]
        # Nếu có 2 công nhân, lấy người nhanh nhất và nhân đôi thời gian
        elif n_workers == 2:
            total_time_for_task = min(worker_times) * 2
        # Nếu có 3 công nhân, lấy người nhanh nhất và nhân ba thời gian
        elif n_workers == 3:
            total_time_for_task = min(worker_times) * 3

        # Lưu trữ thời gian cho từng công đoạn
        stage_times.append({
            'Công Đoạn': df['TÊN CÔNG ĐOẠN'].iloc[task_idx],
            'Tổng Thời Gian': total_time_for_task
        })

    return stage_times

# Sử dụng hàm display_stage_times để hiển thị thời gian của từng công đoạn
stage_times_summary = display_stage_times(best_solution)

# Chuyển thành DataFrame để hiển thị
stage_times_df = pd.DataFrame(stage_times_summary)

# Hiển thị kết quả
stage_times_df

,Công Đoạn,Tổng Thời Gian
0,Xã phà thun lưng - 0,2.027027
1,Lấy dấu cắt gai xù túi sau + gọt góc - 0,2.702703
2,Đo cắt pat túi thân sau - 0,1.351351
3,Đo cắt dây treo gắn túi thân sau - 0,3.378378
4,Nối dây pat túi thân sau - 1,8.620690
...,...,...
80,Vệ sinh công nghiệp - 0,23.484848
81,Lộn quần ( trái sang phải ) - 0,3.378378
82,Mí đáy thân trước - 1,10.135135
83,Lấy dấu lưng thùa khuyu - 0,6.081081


In [ ]:
#tổng thời gian từng công đoạn lớn
#lấy dấu
total_1  =sum(stage['Tổng Thời Gian'] for stage in stage_times_summary[0:4])
#Thân sau
total_2  =sum(stage['Tổng Thời Gian'] for stage in stage_times_summary[4:27])
#Thân trước
total_3  =sum(stage['Tổng Thời Gian'] for stage in stage_times_summary[27:50])
#Lưng
total_4  =sum(stage['Tổng Thời Gian'] for stage in stage_times_summary[50:53])
#quần lót
total_5  =sum(stage['Tổng Thời Gian'] for stage in stage_times_summary[53:57])
#hoàn thành
total_6  =sum(stage['Tổng Thời Gian'] for stage in stage_times_summary[57:85])
# tổng số sản phẩm trong 1 ngày
re_stage_time = total_1+max(max(total_2,total_3)+total_4,total_5)+total_6
print(re_stage_time)
total_product = (28800-re_stage_time)/worker_time_df['Tổng Thời Gian'].max()+1
total_product

1298.4389561975768


291.744910072972

In [ ]:
def display_task_allocation_with_time(individual):
    detailed_task_allocation = {worker: [] for worker in workers}

    for task_idx, worker_list in enumerate(individual):
        task_time = df['TGLT'][task_idx]
        n_workers = len(worker_list)

        # Tính thời gian chia đều lý thuyết cho mỗi công nhân
        theoretical_time_per_worker = task_time / n_workers

        # Điều chỉnh thời gian cho từng công nhân theo độ thông thạo của họ
        for worker in worker_list:
            proficiency = proficiency_levels[worker]
            worker_time = theoretical_time_per_worker / proficiency  # Điều chỉnh thời gian theo độ thông thạo
            detailed_task_allocation[worker].append(f"Công đoạn {df['TÊN CÔNG ĐOẠN'].iloc[task_idx]}: {worker_time:.2f} giây")

    return detailed_task_allocation


# Chọn cá thể tốt nhất (cá thể có độ lệch chuẩn nhỏ nhất)
task_allocation_with_time = display_task_allocation_with_time(best_solution)

# Hiển thị chi tiết công việc cho mỗi công nhân
worker_task_summary = []
worker_time = []

for worker, tasks in task_allocation_with_time.items():
    # Tính tổng thời gian cho mỗi công nhân
    total_time = sum(float(task.split(": ")[1].split()[0]) for task in tasks)
    # Chi tiết các công đoạn
    task_details = " + ".join(tasks)
    worker_task_summary.append({
        'Công Nhân': worker,
        'Chi Tiết Công Việc': task_details,
        'Tổng Thời Gian': total_time
    })
    worker_time.append({
        'Công Nhân': worker,
        'Tổng Thời Gian': total_time
    })

# Chuyển thành DataFrame để hiển thị
worker_task_summary_df = pd.DataFrame(worker_task_summary)

# Hiển thị kết quả chi tiết cho mỗi công nhân
worker_task_summary_df
def display_task_allocation_with_time(individual):
    detailed_task_allocation = {worker: [] for worker in workers}

    for task_idx, worker_list in enumerate(individual):
        task_time = df['TGLT'][task_idx]
        n_workers = len(worker_list)

        # Tính thời gian chia đều lý thuyết cho mỗi công nhân
        theoretical_time_per_worker = task_time / n_workers

        # Điều chỉnh thời gian cho từng công nhân theo độ thông thạo của họ
        for worker in worker_list:
            proficiency = proficiency_levels[worker]
            worker_time = theoretical_time_per_worker / proficiency  # Điều chỉnh thời gian theo độ thông thạo
            detailed_task_allocation[worker].append(f"Công đoạn {df['TÊN CÔNG ĐOẠN'].iloc[task_idx]}: {worker_time:.2f} giây")

    return detailed_task_allocation


# Chọn cá thể tốt nhất (cá thể có độ lệch chuẩn nhỏ nhất)
task_allocation_with_time = display_task_allocation_with_time(best_solution)

# Hiển thị chi tiết công việc cho mỗi công nhân
worker_task_summary = []
worker_time = []

for worker, tasks in task_allocation_with_time.items():
    # Tính tổng thời gian cho mỗi công nhân
    total_time = sum(float(task.split(": ")[1].split()[0]) for task in tasks)
    # Chi tiết các công đoạn
    task_details = " + ".join(tasks)
    worker_task_summary.append({
        'Công Nhân': worker,
        'Chi Tiết Công Việc': task_details,
        'Tổng Thời Gian': total_time
    })
    worker_time.append({
        'Công Nhân': worker,
        'Tổng Thời Gian': total_time
    })

# Chuyển thành DataFrame để hiển thị
worker_task_summary_df = pd.DataFrame(worker_task_summary)

# Hiển thị kết quả chi tiết cho mỗi công nhân
worker_task_summary_df


,Công Nhân,Chi Tiết Công Việc,Tổng Thời Gian
0,Lê Thị Phương Thảo,Công đoạn Lựa số+ bỏ rập đh tt + lấy dấu + pag...,94.59
1,Hoàng Thị Mỹ Yến,Công đoạn Kẹp mí đáp túi + bỏ dây treo thân sa...,88.65
2,Trần Thị Tuyền,Công đoạn Xã phà thun lưng - 0: 1.01 giây + Cô...,92.23
3,Ngô Khắc Hiển,Công đoạn Chốt pat túi thân sau - 1: 7.00 giây...,85.34
4,Hồ Thị Diệp,Công đoạn Chốt dây treo túi sau - 1: 4.89 giây...,81.33
5,Nguyễn Thị Mỹ Nhung,Công đoạn May nhãn cơi túi thân sau - 1: 26.72...,89.08
6,Thái Thị Nga,Công đoạn Lấy dấu cắt gai xù túi sau + gọt góc...,89.96
7,Trần Thị Cúc,Công đoạn Bọ túi thân sau x2 - 3: 10.00 giây +...,82.50
8,Lê Thị Thanh Hà,Công đoạn Nối dây pat túi thân sau - 1: 8.62 g...,85.20
9,Nguyễn Thị Thúy,Công đoạn Lược miệng túi thân sau - 1: 5.46 gi...,89.37


In [ ]:
df['Công Nhân Mới (Nhiều)'] = [", ".join(worker_list) for worker_list in best_solution]

# Hiển thị kết quả cuối cùng
df[['TÊN CÔNG ĐOẠN', 'Công Nhân Mới (Nhiều)']]

,TÊN CÔNG ĐOẠN,Công Nhân Mới (Nhiều)
0,Xã phà thun lưng - 0,"Nguyễn Thị Gấm, Trần Thị Tuyền"
1,Lấy dấu cắt gai xù túi sau + gọt góc - 0,"Thái Thị Nga, Nguyễn Thị Hồng, Bùi Thị Hồng Vân"
2,Đo cắt pat túi thân sau - 0,"Trần Thị Tuyền, Nguyễn Thị Thu"
3,Đo cắt dây treo gắn túi thân sau - 0,"Trần Thị Tuyền, Nguyễn Thị Hồng, Nguyễn Thị Hài"
4,Nối dây pat túi thân sau - 1,Lê Thị Thanh Hà
...,...,...
80,Vệ sinh công nghiệp - 0,"Nguyễn Thị Gấm, Trần Thị Cúc"
81,Lộn quần ( trái sang phải ) - 0,"Thái Thị Nga, Trần Thị Tuyền"
82,Mí đáy thân trước - 1,"Bùi Thị Hồng Vân, Hồ Thị Diệp, Trần Thị Tuyền"
83,Lấy dấu lưng thùa khuyu - 0,"Trần Thị Cẩm Rin, Thái Thị Nga, Nguyễn Thị Hồng"


In [ ]:
# Hàm tính lương và tổng tiền lương cho tất cả công nhân
def calculate_salary(individual, proficiency_levels, dt):
    worker_salaries = {worker: 0 for worker in workers}  # Khởi tạo lương cho mỗi công nhân bằng 0

    for task_idx, worker_list in enumerate(individual):
        # Lấy đơn giá của công đoạn hiện tại
        task_price = dt['ĐƠN GIÁ'][task_idx]
        num_workers = len(worker_list)  # Số công nhân thực hiện công đoạn

        # Chia đơn giá cho số công nhân nếu nhiều người thực hiện
        price_per_worker = task_price / num_workers

        # Tính lương cho từng công nhân tham gia công đoạn
        for worker in worker_list:
            proficiency = proficiency_levels[worker]  # Lấy năng lực của công nhân
            # Lương cho công đoạn này bằng đơn giá chia cho số công nhân, sau đó nhân năng lực
            salary_for_task = price_per_worker * proficiency
            # Cộng lương cho công đoạn này vào tổng lương của công nhân
            worker_salaries[worker] += salary_for_task

    # Tạo DataFrame chứa kết quả lương của từng công nhân
    salary_df = pd.DataFrame.from_dict(worker_salaries, orient='index', columns=['Tiền Lương'])
    salary_df.reset_index(inplace=True)
    salary_df.columns = ['Công Nhân', 'Tổng Tiền Lương']

    # Tính tổng tiền lương của tất cả công nhân
    total_salary = salary_df['Tổng Tiền Lương'].sum()
    print(f"Tổng tiền lương của tất cả công nhân: {total_salary:.2f}")

    return salary_df, total_salary
# Sử dụng hàm để tính lương cho mỗi công nhân và tổng lương
salary_df, total_salary = calculate_salary(best_solution, proficiency_levels, df)

# Hiển thị DataFrame chứa tổng tiền lương cho mỗi công nhân
salary_df

Tổng tiền lương của tất cả công nhân: 15499.19


,Công Nhân,Tổng Tiền Lương
0,Lê Thị Phương Thảo,1099.640000
1,Hoàng Thị Mỹ Yến,626.400000
2,Trần Thị Tuyền,1070.040000
3,Ngô Khắc Hiển,455.333333
4,Hồ Thị Diệp,578.066667
5,Nguyễn Thị Mỹ Nhung,632.393333
6,Thái Thị Nga,641.286667
7,Trần Thị Cúc,436.166667
8,Lê Thị Thanh Hà,603.200000
9,Nguyễn Thị Thúy,632.780000


In [ ]:
best_solution

[['Nguyễn Thị Gấm', 'Trần Thị Tuyền'],
 ['Thái Thị Nga', 'Nguyễn Thị Hồng', 'Bùi Thị Hồng Vân'],
 ['Trần Thị Tuyền', 'Nguyễn Thị Thu'],
 ['Trần Thị Tuyền', 'Nguyễn Thị Hồng', 'Nguyễn Thị Hài'],
 ['Lê Thị Thanh Hà'],
 ['Phan Thị Thu Hoài', 'Nguyễn Thị Gấm'],
 ['Ngô Khắc Hiển'],
 ['Nguyễn Thị Hài', 'Trần Thị Oanh', 'Hồ Thị Diệp'],
 ['Nguyễn Thị Ngọc Lan'],
 ['Phan Thị Thu Hoài', 'Hoàng Thị Nở'],
 ['Ngô Khắc Hiển', 'Lê Thị Thanh Hà', 'Nguyễn Thị Thu'],
 ['Hoàng Thị Mỹ Yến', 'Trần Thị Thương', 'Nguyễn Thị Hài'],
 ['Hồ Thị Diệp'],
 ['Thái Thị Nga'],
 ['Hoàng Thị Mỹ Yến'],
 ['Nguyễn Thị Hài', 'Hoàng Thị Mỹ Yến', 'Ngô Khắc Hiển'],
 ['Lê Thị Như Quỳnh', 'Thái Thị Nga'],
 ['Ngô Khắc Hiển'],
 ['Nguyễn Thị Gấm', 'Đinh Thị Mi', 'Phan Thị Thu Hoài'],
 ['Lê Thị Như Quỳnh', 'Nguyễn Thị Ngọc Lan'],
 ['Nguyễn Thị Thu'],
 ['Lê Thị Thanh Hà'],
 ['Nguyễn Thị Mỹ Nhung'],
 ['Trần Thị Cúc'],
 ['Phan Thị Thu Hoài', 'Nguyễn Thị Gấm'],
 ['Hoàng Thị Mỹ Yến'],
 ['Lê Thị Thanh Hà', 'Nguyễn Thị Thúy', 'Bùi Thị Hồng

In [ ]:
# Hàm tính lương và tổng tiền lương cho tất cả công nhân
def calculate_salary(individual, proficiency_levels, dt):
    worker_salaries = {worker: 0 for worker in workers}  # Khởi tạo lương cho mỗi công nhân bằng 0

    for task_idx, worker_list in enumerate(individual):
        # Lấy đơn giá của công đoạn hiện tại
        task_price = dt['ĐƠN GIÁ'][task_idx]
        num_workers = len(worker_list)  # Số công nhân thực hiện công đoạn

        # Chia đơn giá cho số công nhân nếu nhiều người thực hiện
        price_per_worker = task_price / num_workers

        # Tính lương cho từng công nhân tham gia công đoạn
        for worker in worker_list:
            proficiency = proficiency_levels[worker]  # Lấy năng lực của công nhân
            # Lương cho công đoạn này bằng đơn giá chia cho số công nhân, sau đó nhân năng lực
            salary_for_task = price_per_worker * proficiency
            # Cộng lương cho công đoạn này vào tổng lương của công nhân
            worker_salaries[worker] += salary_for_task

    # Tạo DataFrame chứa kết quả lương của từng công nhân
    salary_df = pd.DataFrame.from_dict(worker_salaries, orient='index', columns=['Tiền Lương'])
    salary_df.reset_index(inplace=True)
    salary_df.columns = ['Công Nhân', 'Tổng Tiền Lương']

    # Tính tổng tiền lương của tất cả công nhân
    total_salary = salary_df['Tổng Tiền Lương'].sum()
    print(f"Tổng tiền lương của tất cả công nhân: {total_salary:.2f}")

    return salary_df, total_salary
# Sử dụng hàm để tính lương cho mỗi công nhân và tổng lương
salary_df, total_salary = calculate_salary(best_solution, proficiency_levels, df)

# Hiển thị DataFrame chứa tổng tiền lương cho mỗi công nhân
salary_df

Tổng tiền lương của tất cả công nhân: 15499.19


,Công Nhân,Tổng Tiền Lương
0,Lê Thị Phương Thảo,1099.640000
1,Hoàng Thị Mỹ Yến,626.400000
2,Trần Thị Tuyền,1070.040000
3,Ngô Khắc Hiển,455.333333
4,Hồ Thị Diệp,578.066667
5,Nguyễn Thị Mỹ Nhung,632.393333
6,Thái Thị Nga,641.286667
7,Trần Thị Cúc,436.166667
8,Lê Thị Thanh Hà,603.200000
9,Nguyễn Thị Thúy,632.780000


In [ ]:
df[['TÊN CÔNG ĐOẠN', 'Công Nhân Mới (Nhiều)']]

,TÊN CÔNG ĐOẠN,Công Nhân Mới (Nhiều)
0,Xã phà thun lưng - 0,"Nguyễn Thị Gấm, Trần Thị Tuyền"
1,Lấy dấu cắt gai xù túi sau + gọt góc - 0,"Thái Thị Nga, Nguyễn Thị Hồng, Bùi Thị Hồng Vân"
2,Đo cắt pat túi thân sau - 0,"Trần Thị Tuyền, Nguyễn Thị Thu"
3,Đo cắt dây treo gắn túi thân sau - 0,"Trần Thị Tuyền, Nguyễn Thị Hồng, Nguyễn Thị Hài"
4,Nối dây pat túi thân sau - 1,Lê Thị Thanh Hà
...,...,...
80,Vệ sinh công nghiệp - 0,"Nguyễn Thị Gấm, Trần Thị Cúc"
81,Lộn quần ( trái sang phải ) - 0,"Thái Thị Nga, Trần Thị Tuyền"
82,Mí đáy thân trước - 1,"Bùi Thị Hồng Vân, Hồ Thị Diệp, Trần Thị Tuyền"
83,Lấy dấu lưng thùa khuyu - 0,"Trần Thị Cẩm Rin, Thái Thị Nga, Nguyễn Thị Hồng"
